# 1.1 Finite Difference Formulation

Prepared by (C) Vasken Dermardiros

This series of documents are intended to show my approach and implementation of the finite difference formulation to solve a system of heat balance equations. This is not the only way to do it, however I do find my method to be rather convenient. In my day to day work, I do seperate the models from the "solvers" and from other scripts that do supervisory type calculations above it, such as model calibration and model-based predictive controls. You'll see that in Chapters 2 & 3.

As of May 2016, this document still remain as a work in progress and is updated on my spare time. Please do inform me of mistakes or bugs or improvements. I am quite open minded in that regard.

### Finite Difference Equation

Heat conduction through a medium is governed by Fourier's Law of heat conduction (or sometimes refered as Fourer's Law of heat diffusion).  
Starting from a generalized case for multi-dimensional heat transfer the heat conduction equation can be reworked into a finite difference approximation of the partial differential equation (Dermardiros, 2015). The analytical solution can seldom be calculated for the majority of cases and an approximation is necessary.

$\rho \displaystyle \frac{\partial h}{\partial t} + \nabla(-k \nabla T) = 0$

Simplified to a 1-dimensional system in the $x$ direction,

$\rho \displaystyle \frac{\partial h}{\partial t} + \frac{\partial}{\partial x}(-k \nabla T) = 0$

Assuming a constant density of the material, by performing the inner derivative and applying the Product Law of calculus, we obtain:

$\rho \displaystyle \frac{dh}{dT}\frac{\partial T}{\partial t} -k \frac{\partial^2 T}{\partial x^2} -\frac{dk}{dT} \left(\frac{\partial T}{\partial x}\right)^2 = 0$

The specific heat, $c_p$, is defined as:

$c_p(T) \equiv \displaystyle \frac{dh}{dT}$

And so,

$\displaystyle \frac{\partial T}{\partial t} = \frac{k}{\rho c_p(T)} \left[ \frac{\partial^2 T}{\partial x^2} + \frac{1}{k} \frac{dk}{dT} \left(\frac{\partial T}{\partial x}\right) ^2 \right]$

For the case where conductivity is constant, then the equation simplifies to:

$\displaystyle \frac{\partial T}{\partial t} = \frac{k}{\rho c_p(T)} \frac{\partial^2 T}{\partial x^2}$

Since the above equation can only be solved analytically for certain cases, a finite difference approximation of the partial differential equation is necessary. Writing the 1st order forward difference equation for the left hand side:

$\displaystyle \left( \frac{\partial T}{\partial t} \right)^t_i = \frac{T^{t+1}_i - T^t_i}{\Delta t} + O(\Delta t)$

Now, the central difference equation for the right hand side 2nd order differential equation:

$\displaystyle \left( \frac{\partial^2 T}{\partial x^2} \right)^{t+\vartheta}_i = \frac{T^{t+\vartheta}_{i+1} - 2T^{t+\vartheta}_i + T^{t+\vartheta}_{i-1}}{\Delta x^2} + O(\Delta x)^2$

$\vartheta$ is between 0 and 1, typically $\vartheta$ is chosen as 0, ½, or 1, which yield, respectively, the Explicit, Crank-Nicholson and Implicit finite difference schemes. In the Explicit case, the future timestep is computed node by node, one by one, in order. The last two yield a system of equations which must be solved simultaneously.

Using the finite difference approximations, and rearranging the equation for the future timestep, we obtain:

$T^{t+1}_i = T^{t}_i + \displaystyle \frac{k \Delta t}{\rho c_p(T) \Delta x^2} \left[ (1-\vartheta)(T^{t}_{i+1} - 2T^{t}_i + T^{t}_{i-1}) + \vartheta (T^{t+1}_{i+1} - 2T^{t+1}_i + T^{t+1}_{i-1}) \right]$

Now, let's look at a case where the neighbouring nodes have a fixed boundary temperature, $T_{bound}$. And assuming the initial temperature at the active node is $T_{init}$ and is at a lower temperature than the neighbouring nodes. For the future timestep, physically, the temperature at the active node cannot exceed the boundary temperature since there is no internal heat generation.

$T_{init} < T_{bound}$

$T^t_{i-1} = T^t_{i+1} = T^{t+1}_{i-1} = T^{t+1}_{i-1} = T_{bound}$

$T^t_i = T_{init}$

$T^{t+1}_i \leq T_{bound}$

And since,

$\displaystyle \frac{k \Delta t}{\rho c_p(T) \Delta x^2} = \frac{T^{t+1}_i-T^t_i}{(1-\vartheta)(T^{t}_{i+1} - 2T^{t}_i + T^{t}_{i-1}) + \vartheta (T^{t+1}_{i+1} - 2T^{t+1}_i + T^{t+1}_{i-1})}$

$\therefore \ \displaystyle \Delta t \leq \frac{\rho c_p(T) \Delta x^2}{2k(1-\vartheta)}$

To assure numerical stability in the solution, the timestep must be chosen according to the previous equation.

For the Explicit case where $\vartheta = 0$:

$\Delta t \leq \displaystyle \frac{\rho c_p(T) \Delta x^2}{2k}$

For the Crank-Nicholson case where $\vartheta = ½$:

$\Delta t \leq \displaystyle \frac{\rho c_p(T) \Delta x^2}{k}$

Finally, for the Implicit case where $\vartheta = 1$:

$\Delta t \leq \infty$

These timesteps assure numerical stability. For the Implicit case, there is no restriction on timestep, however, for very large timesteps, the solution may oscillate. Although the oscillations will eventually dampen, large timesteps do not guarantee physically plausible solutions.

Additionally, these timesteps are for 1-dimensional heat transfer -- which is typically the case for buildings. For 2-d and 3-d heat transfer, the explicit method finite difference method stability criterion will differ. (Its derivation is beyond the scope of this document.) Final note, for 2-d and 3-d heat transfer, the ordering of the nodes may need to be carefully considered to assure the matrix can be inverted is not ill-conditionned.

### Finite Difference Implementation
For a given node $i$, which is: exchanging heat with all neigbouring nodes $j$ and $k$ through conduction, convection and radiation, noted as an equivalent conductance $U$; has capacitance $C$ (thermal mass); and exchanging heat with a source $Q$ the finite difference equation can be written:

$\displaystyle \sum_{j}{[U_{ij}^\theta (T_j^\theta-T_i^\theta)]}+\sum_{k}{[U_{ik}^\theta (T_k^\theta-T_i^\theta)]}-\displaystyle \frac{C(T)_i}{\Delta t} (T_i^{t+1}-T_i^{t}) + \dot{Q}_i^\theta = 0$,  
where,  
$\theta = t \ $: explicit case,  
$\theta = t+1$: implicit case,  
$U_{ij}$: conductance between nodes $i$ and $j$ equal to $\frac{kA}{dx}$ for conductance, $h_{conv}A$ for convection and $h_{rad}A$ for radiation, $\frac{W}{K}$,  
$U_{ik}$: conductance between nodes $i$ and $k$ where node $k$ has a defined or known temperature (boundary condition), $\frac{W}{K}$,  
$C(T)$: capacitance of node $i$ equal to $\rho c_p(T) A dx$, $\frac{J}{K}$,  
$\dot{Q}$: heat flow into the node, $W$, and,  
$\Delta t$: timestep, $s$.

As an example where $i = 1$, we can draw the following thermal network:  
<img src="Figures/fdm.png" width=350 align="left"/>

#### Implicit Case
Generalizing for a system, the equation can be written in a matrix form:

$\begin{bmatrix} \displaystyle
    \sum_j^N{U_{1j}}+\sum_k^M{U_{1k}}+\frac{C_1}{\Delta t} & -U_{12} & \dots  & -U_{1N} \\
    \vdots & \vdots & \ddots & \vdots \\ 
    -U_{N1} & -U_{N2} & \dots  & \displaystyle 
    \sum_j^N{U_{Nj}}+\sum_k^M{U_{Nk}}+\frac{C_N}{\Delta t}
\end{bmatrix}
\begin{Bmatrix} T_1 \\ \vdots \\ T_N \end{Bmatrix}^{t+1} = 
\begin{Bmatrix} \displaystyle \dot{Q}_1 + \sum_k^M{(U_{1kk}T_{kk})}+\frac{C_1}{\Delta t}T_1^t \\ \vdots \\ \displaystyle \dot{Q}_N + \sum_k^M{(U_{Nkk}T_{kk})}+\frac{C_N}{\Delta t}T_N^t \end{Bmatrix}$,  
where,  
$N$ is the number of nodes, and,  
$M$ is the number of nodes with known temperatures.  

#### Explicit Case
Similarly, for the explicit case, the equations can be written in matrix form:

$\begin{Bmatrix} T_1 \\ \vdots \\ T_N \end{Bmatrix}^{t+1} = 
\begin{Bmatrix} \displaystyle \frac{\Delta t}{C_1} \\ \vdots \\ \displaystyle \frac{\Delta t}{C_N} \end{Bmatrix} 
\odot 
\left(
\begin{bmatrix} \displaystyle
    -\sum_j^N{U_{1j}}-\sum_k^M{U_{1k}}+\frac{C_1}{\Delta t} & U_{12} & \dots  & U_{1N} \\
    \vdots & \vdots & \ddots & \vdots \\ 
    U_{N1} & U_{N2} & \dots  & \displaystyle 
    -\sum_j^N{U_{Nj}}-\sum_k^M{U_{Nk}}+\frac{C_N}{\Delta t}
\end{bmatrix}
\begin{Bmatrix} T_1 \\ \vdots \\ T_N \end{Bmatrix}^t + 
\begin{Bmatrix} \displaystyle \dot{Q}_1 + \sum_k^M{(U_{1kk}T_{kk})} \\ \vdots \\ \displaystyle \dot{Q}_N + \sum_k^M{(U_{Nkk}T_{kk})} \end{Bmatrix}
\right),$

where,  
$\odot$ is an element-wise multiplication operator,  
$N$ is the number of nodes, and,  
$M$ is the number of nodes with known temperatures.

There are times, however, when a thermal node will have negligeable thermal capacitance $(C \approx 0)$, sometimes refered to as *arithmetic nodes* -- whereas nodes with capacitance are known as *diffusion nodes*.

Performing an energy balance at the thermal node with negligeable thermal capacitance, the finite difference equation becomes:

$T_i^{t+1} = \displaystyle \frac{\displaystyle \sum_{j}{[U_{ij}^t \ T_j^t]} + \displaystyle \sum_{k}{[U_{ikk}^t \ T_{kk}^t]} + \dot{Q}_i}{\displaystyle \sum_{j}{U_{ij}^t} + \displaystyle \sum_{k}{U_{ikk}^t}}$

(For simplificity's sake, the equation is not written in a vectorized form, however, the scripts use a vectorized implementation.)

---------------
#### Finite Difference Implementation: Seperating Inputs in the Python Script into Matrices
In the Python Script implementing the finite difference method (FDM), the user will need to input all the conductances connecting the thermal nodes together. Since the system is linear, only values of $U_{ij}$ where $j > i$ need to be input due to symmetry. $U = 0$ for nodes not connected to one another. For convenience, the capacitance values $C$ for nodes with thermal mass are input in the vector $\bf{C}$. Connections to known temperature sources are input in the matrix $\bf{F}$. Finally, heat flow into the nodes are input in the vector $\bf{Q}_{input}$.

$\textbf{U}_{input} = \begin{bmatrix}
    0 & U_{12} & U_{13} & \dots  & U_{1N} \\
    0 & 0 & U_{23} & \dots  & U_{2N} \\
    0 & 0 & 0 & \dots  & U_{3N} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\ 
    0 & 0 & 0 &  \dots  & 0
\end{bmatrix},
\ \textbf{C} = \begin{Bmatrix} C_1 \\ C_2 \\ C_3 \\ \vdots \\ C_N \end{Bmatrix}, 
\ \dot{ \textbf{Q} }_{input} = \begin{Bmatrix} \dot{Q}_1 \\ \dot{Q}_2 \\ \dot{Q}_3 \\ \vdots \\ \dot{Q}_N \end{Bmatrix}, 
\ \textbf{T}_K = \begin{Bmatrix} T_{k1} \\ T_{k2} \\ T_{k3} \\ \vdots \\ T_{kM} \end{Bmatrix},
\ \textbf{F} = \begin{bmatrix}
    U_{1k1} & U_{1k2} & U_{1k3} & \dots  & U_{1kM} \\
    U_{2k1} & U_{2k2} & U_{2k3} & \dots  & U_{2kM} \\
    U_{3k1} & U_{3k2} & U_{3k3} & \dots  & U_{3kM} \\
    \vdots & \vdots & \vdots & \ddots & \vdots \\ 
    U_{Nk1} & U_{Nk2} & U_{Nk3} & \dots  & U_{NkM} \\
\end{bmatrix}. $

Where,  
$U_{ij} = \displaystyle \left( \frac{1}{hA} \right)^{-1}, $ for convection  
$U_{ij} = \displaystyle \left( \frac{1}{UA} \right)^{-1} or \ \left( \frac{\Delta x}{kA} \right)^{-1}, $ for conduction  
$U_{ij} = \displaystyle \left( ACH*Volume*\rho_{air}*c_{p, air}*3600^{-1} \right), $ for infiltration, and  
$U_{ij} = \displaystyle \left( \frac{1}{h_{out}A} + \frac{\Delta x}{kA} + \frac{1}{h_{in}A} \right)^{-1}, $ for combined cases.

$U_{12}$ in $\textbf{U}_{input}$ is the conductance between nodes 1 and 2. Nodes 1 and 2 have temperatures $T_1$ and $T_2$ which we are solving for.  
$U_{1k1}$ in $\textbf{F}$ is the conductance between node 1 and external source 1: $T_{k1}$. The external source has a predetermined/known/set temperature profile.

---------
# Reference
Dermardiros, Vasken (2015) *Modelling and Experimental Evaluation of an Active Thermal Energy Storage System with Phase-Change Materials for Model-Based Control.* Masters thesis, Concordia University. Available: http://www.bit.ly/VDermMascThesis